In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient
from slugify import slugify
from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import time


In [2]:
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [3]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    #print r["data"][0]
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [4]:
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "http://localhost:3000"
#TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"


In [5]:

# data
title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}

# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()
for artist in db.minedArtists.find({"name":"Foo Fighters"}) :
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    tours = artist.get("tours")    
    singlegigs = artist.get("singleGigs")
    dateini=0
    touring=0
    tour_indice=0
    tmpOLDgig ={}
    for tour in tours:
        tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
        print tour_color
        #print tour
        gig_indice =0
        dates = tour.get("gigs")
        for date in dates:
            #print dates[gig_indice]
            #print date.get("datetime")
            venue = date.get("venue")
            latitude = venue.get("latitude")
            longitude = venue.get("longitude")
            #print venue
            ###concat key
            idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
            
            if idd not in my_nodesdict:
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)
                    }
            else:
                
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(my_nodesdict[idd]["data"].get("weight"))+1 ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight"))+1))+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color
                    
                    }
            if dateini == 0:
                dateini =1
            else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    "notes" : "distance : "+str(round(dates[gig_indice]["distanceToNextGig"],2))+" km  \n"+"source : "+my_nodes[-1]["data"]["id"]+"  target : "+node["id"]+"  \ndatesource : "+dates[gig_indice -1]["datetime"].isoformat()+"  \ndatetarget : "+dates[gig_indice]["datetime"].isoformat()+"  \ngroup:"+str(tour_indice),
    #               "weight" : float(e["weight"])
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    "group" : str(tour_indice)
                    
                }
                print edge
                my_edges.append({ "data" : edge })
            my_nodes.append({ "data" : node })   
            my_nodesdict[node["id"]]= { "data" : node }
            gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
        tour_indice +=1
        #print "coords",latitude,"/",longitude

Foo Fighters
#7f3f3f
1
{'end': '1995-03-03T19:00:00', 'target': 'satyricon45-122', 'start': '1995-02-19T19:00:00', 'color': '#7f3f3f', 'notes': 'distance : 1323.85 km  \nsource : boat-house47-122  target : satyricon45-122  \ndatesource : 1995-02-19T19:00:00  \ndatetarget : 1995-03-03T19:00:00  \ngroup:0', 'source': 'boat-house47-122', 'group': '0'}
2
{'end': '1995-03-10T19:00:00', 'target': 'spaceland34-118', 'start': '1995-03-03T19:00:00', 'color': '#7f3f3f', 'notes': 'distance : 2126.3 km  \nsource : satyricon45-122  target : spaceland34-118  \ndatesource : 1995-03-03T19:00:00  \ndatetarget : 1995-03-10T19:00:00  \ngroup:0', 'source': 'satyricon45-122', 'group': '0'}
#7f523f
0
{'end': '1995-04-17T19:00:00', 'target': 'bottleneck38-95', 'start': '1995-05-19T19:00:00', 'color': '#7f523f', 'notes': 'distance : 440.42 km  \nsource : spaceland34-118  target : bottleneck38-95  \ndatesource : 1995-05-19T19:00:00  \ndatetarget : 1995-04-17T19:00:00  \ngroup:1', 'source': 'spaceland34-118', '

4
{'end': '2015-07-19T19:00:00', 'target': 'fenway-park42-71', 'start': '2015-07-18T20:00:00', 'color': '#4c3f7f', 'notes': 'distance : 10608.0 km  \nsource : fenway-park42-71  target : fenway-park42-71  \ndatesource : 2015-07-18T20:00:00  \ndatetarget : 2015-07-19T19:00:00  \ngroup:74', 'source': 'fenway-park42-71', 'group': '74'}
5
{'end': '2015-07-24T10:00:00', 'target': 'naeba-ski-resort37139', 'start': '2015-07-19T19:00:00', 'color': '#4c3f7f', 'notes': 'distance : 7725.03 km  \nsource : fenway-park42-71  target : naeba-ski-resort37139  \ndatesource : 2015-07-19T19:00:00  \ndatetarget : 2015-07-24T10:00:00  \ngroup:74', 'source': 'fenway-park42-71', 'group': '74'}
#5f3f7f
0
{'end': '2015-08-12T19:00:00', 'target': 'rexall-place53-113', 'start': '2015-08-29T17:30:00', 'color': '#5f3f7f', 'notes': 'distance : 284.78 km  \nsource : naeba-ski-resort37139  target : rexall-place53-113  \ndatesource : 2015-08-29T17:30:00  \ndatetarget : 2015-08-12T19:00:00  \ngroup:75', 'source': 'naeba-

In [6]:
print len(my_nodesdict)

599


In [22]:
print my_nodesdict

{'concert-hall30-103': {'data': {'start': '1996-04-03T19:00:00', 'end': '1996-04-04T19:00:00', 'name': 'concert-hall', 'weight': 1.0, 'lat': 30.34, 'lng': -103.7502778, 'notes': 'name : concert-hall  \nlat : 30.34  \nlng :-103.7502778  \n weight :1  \nstart : 1996-04-03T19:00:00  \nend : 1996-04-04T19:00:00  \ncolor :#727f3f', 'id': 'concert-hall30-103', 'color': '#727f3f'}}, 'santa-barbara-bowl34-119': {'data': {'start': '1996-05-03T19:00:00', 'end': '2006-09-03T19:00:00', 'name': 'santa-barbara-bowl', 'weight': 2.0, 'lat': 34.432517, 'lng': -119.69369, 'notes': 'name : santa-barbara-bowl  \nlat : 34.432517  \nlng :-119.69369  \n weight :2.0  \nstart : 1996-05-03T19:00:00  \nend : 2006-09-03T19:00:00  \ncolor :#7f3f79', 'id': 'santa-barbara-bowl34-119', 'color': '#7f3f79'}}, 'koengen605': {'data': {'start': '2008-06-11T19:00:00', 'end': '2008-06-12T19:00:00', 'name': 'koengen', 'weight': 1.0, 'lat': 60.4009895, 'lng': 5.3176599, 'notes': 'name : koengen  \nlat : 60.4009895  \nlng :5.3

In [23]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)
my_nodesdictList = []
for key, value in my_nodesdict.iteritems():
    
    my_nodesdictList.append(value)



ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'iJh0dqmM3urFMtm3IEDW_BMSPVax37cx3HABaVmIcc8', u'userId': u'MSreQEjksjoK2igp5'}}


In [24]:
print my_nodesdictList

[{'data': {'start': '1996-04-03T19:00:00', 'end': '1996-04-04T19:00:00', 'name': 'concert-hall', 'weight': 1.0, 'lat': 30.34, 'lng': -103.7502778, 'notes': 'name : concert-hall  \nlat : 30.34  \nlng :-103.7502778  \n weight :1  \nstart : 1996-04-03T19:00:00  \nend : 1996-04-04T19:00:00  \ncolor :#727f3f', 'id': 'concert-hall30-103', 'color': '#727f3f'}}, {'data': {'start': '1996-05-03T19:00:00', 'end': '2006-09-03T19:00:00', 'name': 'santa-barbara-bowl', 'weight': 2.0, 'lat': 34.432517, 'lng': -119.69369, 'notes': 'name : santa-barbara-bowl  \nlat : 34.432517  \nlng :-119.69369  \n weight :2.0  \nstart : 1996-05-03T19:00:00  \nend : 2006-09-03T19:00:00  \ncolor :#7f3f79', 'id': 'santa-barbara-bowl34-119', 'color': '#7f3f79'}}, {'data': {'start': '2008-06-11T19:00:00', 'end': '2008-06-12T19:00:00', 'name': 'koengen', 'weight': 1.0, 'lat': 60.4009895, 'lng': 5.3176599, 'notes': 'name : koengen  \nlat : 60.4009895  \nlng :5.3176599  \n weight :1  \nstart : 2008-06-11T19:00:00  \nend : 200

In [26]:
title = "Foo Fighters test edges and notes 4"
create_topogram(title, my_nodesdictList, my_edges)

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


Creating topogram 'Foo Fighters test edges and notes 4'
> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'Foo Fighters test edges and notes 4', u'userId': u'MSreQEjksjoK2igp5', u'_id': u'XgTJffgaKK2YsJaFu', u'slug': u'foo-fighters-test-edges-and-notes-4', u'createdAt': u'2017-10-28T08:06:52.606Z'}, u'statusCode': 200}
topogram ID : XgTJffgaKK2YsJaFu
{u'status': u'success', 'status_code': 200, u'data': [u'Ys6e66xhFea5zE48P', u'XCh4jHdwxHDqHZzsL', u'gqsWqJ8u2A7qcezow', u'ydMwv5BkxbSFRui5w', u'CbMpzNpGjzAbYPnmP', u'qwGEM4orFtrhSjHkb', u'HY8XeDxxbHa8RXi8e', u'izdQ4kqXwYSwTaSra', u'MJsC3DQrBWr8nua3D', u'TrpF7wtMxmBRxAiCN', u'zwDCYnZ5RLMFPYyRS', u'qtdGf657Pg6BsNyGh', u'ZziSJy487f3erQKuD', u'caNCo5yf46AiniNjP', u'Rn37b9aQFCH3sQcyC', u'MD9DqwvPDXSuyosWE', u'Q7jqqPoTcFXNpEvKe', u'uHzdo6GH3WbY4HTBD', u'CegXL9tsWuY3ePfKS', u'78L8duerg8NqYuoYa', u'nSYMzWEbqiRJy5CF8', u'bT6mBETPeDrifti8K', u'KL4oEDESpqGRBA8oE', u'RFTKnPFdLFSuqM8wa', 

852 edges created.
done. Topogram has been updated. Check it at http://localhost:3000/topograms/XgTJffgaKK2YsJaFu
